In [ ]:
from pathlib import Path
import json

In [ ]:
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
plt.ioff()

In [ ]:
model_frontal = '../models/haarcascade_frontalface_default.xml'
model_profile = '../models/haarcascade_profileface.xml'

In [ ]:
sample_parent = '../data/wiki_crop/00/'
output_parent = '../data/wiki_crop_opencv_position_detection/'

sample_frontal_face = [
    '69300_1950-05-11_2009.jpg',
    '196900_1884-05-20_1940.jpg',
    '346200_1968-12-09_2005.jpg',
    '487200_1890-12-11_1964.jpg',
    '489500_1970-05-05_2006.jpg'
]

sample_profile_face = [
    '23300_1962-06-19_2011.jpg',
    '102100_1970-10-09_2008.jpg',
    '654800_1950-01-03_1987.jpg',
    '681100_1959-10-10_2010.jpg'
    
]

sample_contest_face = [
    '262800_1943-04-06_2011.jpg',
    '102100_1970-10-09_2008.jpg',
    '413100_1964-09-14_1994.jpg',
    '575600_1979-07-13_2013.jpg',
    '634600_1936-10-11_1978.jpg'
    
]

In [ ]:
for i, _ in enumerate(sample_frontal_face):
    if isinstance(sample_frontal_face[i], Path):
        continue
        
    sample_frontal_face[i] = Path(sample_parent + sample_frontal_face[i])
    
for i, _ in enumerate(sample_profile_face):
    if isinstance(sample_profile_face[i], Path):
        continue
    
    sample_profile_face[i] = Path(sample_parent + sample_profile_face[i])
    
for i, _ in enumerate(sample_contest_face):
    if isinstance(sample_contest_face[i], Path):
        continue
    
    sample_contest_face[i] = Path(sample_parent + sample_contest_face[i])

In [ ]:
image_cv2 = cv2.imread(str(sample_frontal_face[0]))

In [ ]:
face_classifier_frontal = cv2.CascadeClassifier(model_frontal)
face_classifier_profile = cv2.CascadeClassifier(model_profile)

In [ ]:
def detect_for_picture(file_name, dataset_name):
    image_cv2 = cv2.imread(str(file_name))
        
    faces_frontal = face_classifier_frontal.detectMultiScale(image_cv2)
    faces_profile = face_classifier_profile.detectMultiScale(image_cv2)

    # draw bounding box for each face detected
    for (x, y, w, h) in faces_frontal:
        color = (255,0,0) # red
        stroke = 5
        cv2.rectangle(image_cv2, (x, y), (x + w, y + h), color, stroke)
    
    for (x, y, w, h) in faces_profile:
        color = (0, 255, 255) # in light blue
        stroke = 5
        cv2.rectangle(image_cv2, (x, y), (x + w, y + h), color, stroke)
    
    plt.figure(figsize=(20,10))
    plt.imshow(image_cv2)
    plt.axis('off')
    
    output_path = Path(output_parent) / Path(dataset_name) / Path(file_name).name
    output_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(output_path)

In [ ]:
for image in sample_frontal_face:
    detect_for_picture(image, 'frontal_face')
    
for image in sample_profile_face:
    detect_for_picture(image, 'profile_face')

for image in sample_contest_face:
    detect_for_picture(image, 'contest_face')

# Process all images

In [ ]:
sample_parent = '../data/wiki_crop/'
sample_output = '../data/wiki_crop_opencv_position_detection/'

In [ ]:
def detect_for_picture_json(file_name):
    image_cv2 = cv2.imread(str(file_name))
        
    faces_frontal = face_classifier_frontal.detectMultiScale(image_cv2)
    faces_profile = face_classifier_profile.detectMultiScale(image_cv2)

    # draw bounding box for each face detected
    faces_frontal = faces_frontal.tolist() if len(faces_frontal) > 0 else []
    faces_profile = faces_profile.tolist() if len(faces_profile) > 0 else []
    
    faces_frontal_result = [[x, y, x + w, y + h] for (x, y, w, h) in faces_frontal]
    faces_profile_result = [[x, y, x + w, y + h] for (x, y, w, h) in faces_profile]
    
    return faces_frontal_result, faces_profile_result

In [ ]:
(Path(sample_output) / 'json').mkdir(parents=True, exist_ok=True)

for dataset in Path(sample_parent).iterdir():
    detection_result = {}
    
    images = list(dataset.glob('**/*.jpg'))
    for image in tqdm(images):
        faces_frontal, faces_profile = detect_for_picture_json(image)
        detection_result[image.name] = {
            'faces_frontal': faces_frontal,
            'faces_profile': faces_profile
        }

    with open(Path(sample_output) / 'json' / f'{dataset.name}.json', 'w', encoding='utf-8') as f:
        json.dump(detection_result, f, ensure_ascii=False, indent=4)